# Práctica 1 - Self-Organising Maps - COLORES
## Introducción

Un SOM es un tipo de red neuronal no supervisada planteada por el Profesor Teuvo Kohonen en los 80 inspirada en modelos
biológicos de sistemas neuronales. Se utilizan para mapear datos multidimensionales en menores dimensiones permitiendo
reducir problemas complejos a simples mediantes técnicas de reducción de dimensionalidad y clusterización.

La mayor desventaja de un SOM es que requiere que los pesos de las neuronas se hayan entrenado de manera suficiente
para poder clasificar las entradas. Por tanto, cuando un SOM recibe muy poca información o una gran cantidad de información
redundante, las agrupaciones resultantes pueden no resultar completamente correctos.

---

Describamos un SOM con un ejemplo. Obtenemos la siguiente tabla como **valores de entrada** con 3 dimensiones y queremos
oganizarla en un mapa bidimensional:

|| R | G | B |-|| R | G | B |
| :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
|1| 0 | 0 | 0 |-|5| 255 | 255 | 0 |
|2| 255 | 0 | 0 |-|6| 255 | 0 | 255 |
|3| 0 | 255 | 0 |-|7| 0 | 255 | 255 |
|4| 0 | 0 | 255 |-|8| 255 | 255 | 255 |

Entonces, la **red neuronal** serviría como mapa bidimensional, continiendo en cada una de sus posiciones una neurona con su vector
de pesos (representando este un valor RGB). Este vector de pesos determinará la neurona ganadora para cada entrada de la tabla y la
forma en la que actualizarse, según su posición.

El resultado en este caso debería agrupar los valores con un solo 255 (o ninguno) alejados entre ellos y los valores con varios 255
cerca de las neuronas de los grupos que le componen (por ejemplo, la quinta entrada se encontrará entre la neurona que represente a la
segunda entrada y la tercera entrada).

## Origen

El algoritmo SOM surgió de los primeros modelos de redes neuronales, especialmente modelos de memoria asociativa y aprendizaje adaptativo (cf. Kohonen 1984). Un nuevo incentivo fue explicar la organización espacial de las funciones del cerebro, como se observa especialmente en la corteza cerebral. Sin embargo, el SOM no fue el primer paso en esa dirección: hay que mencionar al menos los detectores lineales espacialmente ordenados de von der Malsburg (1973) y el modelo de campo neural de Amari (1980).

Sin embargo, el poder de autoorganización de estos primeros modelos era bastante débil. La invención crucial de Kohonen fue introducir un modelo de sistema que se compone de al menos dos subsistemas que interactúan de diferentes naturalezas. Uno de estos subsistemas es una red neuronal competitiva que implementa la función de que el ganador se lo lleva todo, pero también existe otro subsistema que es controlado por la red neuronal y que modifica la plasticidad sináptica local de las neuronas en el aprendizaje. El aprendizaje se restringe espacialmente a la vecindad local de las neuronas más activas. El subsistema de control de plasticidad podría estar basado en interacciones neuronales no específicas, pero lo más probable es que sea un efecto de control químico. Solo mediante la separación de la transferencia de señales neuronales y el control de la plasticidad ha sido posible implementar un sistema de autoorganización eficaz y robusto.

No obstante, el principio SOM también se puede expresar matemáticamente en forma puramente abstracta, sin referencia a ningún componente neural subyacente o de otro tipo. La primera área de aplicación del SOM fue el reconocimiento de voz (ver Figura 2). En su forma abstracta, el SOM se ha generalizado en el análisis y la exploración de datos, que es para lo que lo utilizaremos nosotros.

## Componentes

Un *SOM* está compuesto por 2 capas:

- Una **capa de entrada** monodimensional con N neuronas, donde N es el número de componentes de los vectores que se le presentan a la red.
- Una **capa de Kohonen** (capa de salida) una capa multidimensional interconectada completamente con la capa anterior cuyas conexiones determinan que neurona se activa y, por tanto, que neurona se asocia a ese patron.

<img src="https://cybernetist.com/images/som_representation.jpg" alt="Capas" width="500" style="display: block; margin: 0 auto; border: solid 5px black"/>

<br>

A la hora de implementar estas capas, deberemos considerar los siguientes elementos:

- Una `matriz multidimensional de neuronas` que procesa patrones que le llegan desde un *array de neuronas de entrada* (tantas entradas como componentes tienen los patrones).
- Una `función discriminante` calculada a partir de un patrón de entrada y los pesos de cada una de las neuronas de la matriz (por ejemplo, la distancia euclídea).
- Un `mecanismo de competición` que compara las funciones discriminantes de todas las neuronas y selecciona aquella con el mejor valor de la función (BMU).
- Un `mecanismo de cooperación` por interacción local que active simultáneamente la neurona seleccionada y sus *vecinas*.
- Un `mecanismo de adaptación` en virtud del cual los parámetros de las neuronas activas aumentan su *función discriminante* en relación a la entrada actual.

## Algoritmo

El algoritmo sobre el que se basa la implementación de un SOM; es decir, los pasos a seguir, es el siguiente:

```markdown
Inicializamos los pesos
Para 1 hasta N iteraciones:
    Seleccionamos el ejemplo sobre el que entrenar
    Calculamos la neurona ganadora
    Actualizamos el vector de dicha neurona
Clusterizamos (clasificamos) el resultado
```
---

**`Inicializar pesos`**

Podemos inicializar los pesos con valores aleatorios que, además, se pueden normalizar mediante:

> $$w_{ij}=\frac{w'_{ij}}{[\sum_{i=1}^{N}(w'_{ij})^2]^{1/2}}$$

**`Calcular la neurona ganadora`**

Para calcular la neurona ganadora, utilizamos una función discriminante con forma:

> $$D_{j} = \left \| X - W_{j} \right \| = \sqrt{ \sum_{i=1}^{N}( x_{i} - w_{ij} )^2}$$

Que nos permite calcular la distancia euclidea entre la entrada y cada una de las neuronas de la capa de Kohonen. En caso que los vectores estén normalizados con la norma euclidea esta sería:

> $$E = XW_{j} = \sum_{i=1}^{N}( x_{i}w_{ij} )$$

Una vez tengamos todas las distancias, activamos aquella neurona que se encuentre más cercana a la entrada; siendo por tanto una salida binaria (se activa o no):

> $$O_{j} =  \begin{cases} & 1\text{ if } x=\min(D_{j}) \\  & 0\text{ if } x\neq\min(D_{j})\end{cases}$$
> $$ para\,\, datos\,\,sin\,\,normalizar\,\,y $$
> $$O_{j} =  \begin{cases} & 1\text{ if } x=\max(E_{j}) \\  & 0\text{ if } x\neq\max(E_{j})\end{cases}$$
> $$\,\, para\,\, datos\,\, normalizados$$

**`Actuaizar la neurona`**

La neurona activada en el caso anterior será ajustada acorde a un coeficiente de aprendizaje, $\eta$; según la Regla de Aprendizaje Competitivo, $\Delta W_{ij}$, que se aplica sobre el vector de pesos de la neurona ganadora como:

> $$\Delta W_{ij} = \eta(t)(X_{i}-W_{ij})$$

Una vez ajustados los pesos de la neurona ganadora, debemos ajustar los pesos de su *vecindario*. Para ello deberemos determinar qué neuronas modificar; es decir, cuales pertenecen al *vecindario*, y cuánto debemos modificarlas. Podemos encontrar las neuronas vecinas mediante cuadrados concéntricos, rombos, u otros polinomios; también podemos hacerlo mediante funciones Gaussianas o Sombrero Mexicano.

A la hora de modificar los pesos de las neuronas, la función que se utilize, normalmente, modifica en menor manera los pesos de la neurona cuanto más nos alejamos de la ganadora; haciendo que las neuronas del *vecindario* más cercanas a la ganadora sean modificadas hacia la entrada más que aquellas que se encuentren más alejadas, creando agrupaciones de neuronas en el resultado final.

A la hora de implementar este proceso, modificaremos la función que calcula el cambio de los pesos de la neurona para que cuente con la siguiente función de amortiguación:

> $$a_{j}=e^\frac{d_{j}^2}{2v_{t}^2}$$

Quedando por tanto, la siguiente ecuación de aprendizaje para las neuronas que no sean la BMU; es decir, todas menos la ganadora.

> $$W_{ij}^{t+1}=W_{ij}^{t}+\eta(t)a_{j}(X_{j}-W_{ij}^{t})$$

**`Cambios por epoch (iteración)`**

- Variación del coeficiente de aprendizaje: $\eta(t)=\eta_{0}(1-\frac{t}{t_{f}})$
- Variación del vecindario: $v(t) = 1+v_{0}(1-\frac{t}{t_{f}})$
- Amortiguación: al depender del valor del vecindario ($v{t}$) variará con cada iteración

# Refuerzo

Una vez completado el proceso de aprendizaje podemos repetirlo reduciendo el learning rate ($\eta$) mediante la multiplicación del mismo por un parámetro $\eta_{shrink}$. A su vez, ampliaremos el valor del número de iteraciones ($t_{f}$) multiplicándolo por un parámetro $T_{factor}$; quedando por tanto que, cada vez que repitamos el proceso, deberemos hacer:

> $$\eta_{o} = \eta_{o}\,\eta_{shrink}$$
> $$y$$
> $$p = p\,T_{factor}$$


## Preparación de entorno
### Importar librerías de código


In [ ]:
# Asegurar las versiones necesarias de Jupyter Lab
#!conda install "jupyterlab>=3" "notebook>=5.3"
#!conda install -c anaconda ipywidgets==7.0
#!conda install ipykernel
#!conda install ipympl
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!jupyter labextension install jupyter-matplotlib
#!jupyter nbextension enable --py widgetsnbextension
from ipywidgets import interact

# Librería Numérica
import numpy as np
# Tipado de datos
from numpy import dtype, floating
from typing import Tuple, Union, List, Any

# Datasets y Dataframes
import pandas as pd

# Preprocesamiento de datos
from sklearn.preprocessing import MinMaxScaler

# Librería para Visualizar Datos
from matplotlib import pyplot as plt
from matplotlib import patches as patches
from matplotlib import cm
%matplotlib notebook

# Librería para el uso de gráficos interactivos
#!conda install -c conda-forge -c plotly jupyter-dash
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

# Librería para aumentar la velocidad de ejecución
#!conda install numba
import numba
from numba import jit

# Eliminar Alertas y Errores Innecesarios
import logging
logger = logging.getLogger("numba")
logger.setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: ignored

#### Dataset que se va a utilizar para el entrenamiento

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Countries2010.csv to Countries2010 (2).csv


In [ ]:
df = pd.read_csv("Countries2010.csv")
pd.set_option('display.max_rows', df.shape[0]+1)
scaler = MinMaxScaler()

datos = scaler.fit_transform(df.drop(['CountryName'], axis=1))
n_paises = datos.shape[0]
n_caracteristicas = datos.shape[1]
df.head(n_paises)

Como podemos ver en lo que mostramos por pantalla, creamos una matriz de *3 x 100* con diferentes valores al azar dentro del rango del código RGB.

Además podemos ver que cada columna representa un color diferente; mientras que cada fila representa, correspondientemente, el elemento R (rojo), G (verde) y B (azul) de cada color.

## SOM Setup
#### Variables definidas por el alumno

In [ ]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
lado_mapa = 10
# Número total de presentaciones, donde una presentación se basa en la aplicación de
# un vector patrón a la entrada de la red
periodo = n_paises*30 # Asegurar que todos los paises se toman el mismo numero de veces (para tener un entrenamiento más homogéneo)
# Coeficiente de aprendizaje inicial que nos servirá para calcular el BMU (Best
# Matching Unit).Cambiará ligeramente cada iteración y eventualmente será 0
# (eta >>> 0).
learning_rate = 0.28

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [ ]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar.
num_datos_entreno = int(np.floor(n_paises*0.8))
datos_entrenamiento = datos[:num_datos_entreno]
datos_clasificar = datos[num_datos_entreno:]
# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = int(lado_mapa/2)
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
matriz_pesos = np.random.random((lado_mapa, lado_mapa, n_caracteristicas))

#### Funciones para entrenar/clasificar

In [ ]:
# Función para encontrar la distancia euclidea
"""
   Encuentra la distancia euclidea entre 2 vectores
   Entradas: (patron_de_entrada, vector_de_pesos)
   Salidas:  (d_euclidea) distancia euclidea entre ambos vectores
"""
#@jit(nopython=True)
def conseguir_distancia_euclidea(v_pesos, p_entrada):
   # Calculamos la distancia con cada uno de los elementos de la matriz de pesos
   # sabiendo que los vectores estan normalizados. Devolvemos el valor de la raiz cuadrada del sumatorio
   return np.sqrt(np.sum((p_entrada-v_pesos)**2))


# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
   # Viajamos a través de la matriz de pesos
   bmu_idx = [0,0]
   bmu_dist = conseguir_distancia_euclidea(m_pesos[0,0], patron_entrada)
   for i in range(1, m):
      for j in range(1, m):
         temp = conseguir_distancia_euclidea(m_pesos[i,j], patron_entrada)
         # Comparamos para obtener la menor de las distancias de los vectores peso
         if temp < bmu_dist:
            # Cuando encontramos una distancia menor, la almacenamos
            bmu_dist = temp
            bmu_idx = [i,j]
   bmu = m_pesos[bmu_idx[0],bmu_idx[1]]
   # Devolvemos el valor del vector acorde de la matriz de pesos y su posición
   return (bmu, bmu_idx)

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
@jit(nopython=True)
def variacion_learning_rate(lr_inicial:float, i:int, n_iteraciones:int) -> float:
   # Calculamos la variación del coeficiente de aprendizaje según la iteración
   # en la que estemos
   learning_rate = lr_inicial * (1 - (i/n_iteraciones))
   # Devuelvo el valor del learning rate en esta iteración
   return learning_rate

In [ ]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  vecindario para la iteración i

"""
@jit(nopython=True)
def variacion_vecindario(vecindario_inicial:float, i:int, n_iteraciones:int) -> float:
   # Calculamos la variación del vecindario según la iteración en la que estemos
   vecindario = 1 + vecindario_inicial * (1 - (i/n_iteraciones))
   # Devuelvo el vecindario de la iteración
   return vecindario

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
@jit(nopython=True)
def decay(distancia_BMU:float, vecindario_actual:int) -> float:
   return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

#### Funciones para dibujar la salida de la red

In [ ]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores:List[int], iter:int) -> None:
    fig = plt.figure()

    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('SOM después de %d iteraciones' % iter)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1, facecolor=matriz_valores[x-1,y-1,:], edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [ ]:

# Entrena la red con el dataset de entrenamiento
# Competición - Comparar las funciones discriminantes
# Cooperación - Activar neurona seleccionada y vecinas simultáneamente
# Adaptación - Aumentar la función discriminante según la entrada actual
# y los parámetros de las neuronas activas
# Bucle principal
@jit
def actualizar_pesos(m_p, lado, p_e, iter, v_actual, bmu_i):
        for x in range(0, lado):
                for y in range(0, lado):
                        # Calculamos la distancia euclidea 2D
                        euc_2D = conseguir_distancia_euclidea(np.array(bmu_i), np.array([x,y]))
                        if euc_2D <= v_actual:
                                # Nuevos pesos
                                m_p[x,y] = m_p[x,y] + variacion_learning_rate(learning_rate, iter, periodo-1) * decay(euc_2D , v_actual) * (p_e - m_p[x,y])
        return m_p
"""
        def err_cuantificacion(num_e:int, patrones_e:np.ndarray[Any, dtype[np.float64]], matriz_p:np.ndarray[Any, dtype[np.float64]], ld_mapa:int) -> float:
                sum = 0
                for i in range(0,num_e):
                        sum += conseguir_distancia_euclidea(np.array(calcular_bmu(patrones_e[i], matriz_p, ld_mapa)[0]), np.array(patrones_e[i]))
                return sum/num_e
"""
def err_topologico(num_e, patrones_e, matriz_p, ld_mapa, n_caract):
        sum = 0
        for i in range(0,num_e):
                bmu_1 = calcular_bmu(patrones_e[i], matriz_p, ld_mapa)[1]
                m_temp = np.copy(matriz_p) #
                m_temp[bmu_1[0], bmu_1[1]] = np.ones(n_caract)*2
                bmu_2 = calcular_bmu(patrones_e[i], m_temp, ld_mapa)[1]
                if np.any(abs(np.array(bmu_1)-np.array(bmu_2)) > 1):
                        sum += 1
        return sum/num_e

# Cambiamos la forma de la matriz
def entrenar_SOM(m_pesos, n_entradas, entradas):
        for i in range(periodo+1):
                # Entrada actual
                entrada = entradas[i%n_entradas]; # Para tomar una entrada aleatoria, usar random.choice(datos)
                # Best Matching Unit
                __, bmu_idx = calcular_bmu(entrada, m_pesos, lado_mapa)
                # Actualizar pesos
                m_pesos = actualizar_pesos(m_pesos, lado_mapa, entrada, i, variacion_vecindario(vecindario, i, periodo-1), bmu_idx)
        return m_pesos

In [ ]:
matriz_pesos = entrenar_SOM(matriz_pesos, n_paises, datos)
print(matriz_pesos)

[[[0.58550107 0.02098705 0.00274243 ... 0.04991017 0.02422427 0.23943231]
  [0.60350385 0.01653062 0.00223146 ... 0.04392779 0.01978834 0.20977395]
  [0.60270162 0.00619095 0.00126899 ... 0.0304166  0.01098389 0.12581033]
  ...
  [0.38635357 0.04651596 0.01034911 ... 0.24851595 0.07990076 0.14308738]
  [0.34256604 0.07499346 0.038325   ... 0.35762118 0.10689121 0.16146766]
  [0.32631452 0.09032023 0.04927303 ... 0.38814007 0.11963021 0.19189241]]

 [[0.59063121 0.01668115 0.00234635 ... 0.05294623 0.02021545 0.2081004 ]
  [0.59214607 0.01052379 0.00153027 ... 0.04967238 0.01484202 0.16390178]
  [0.56695013 0.00514949 0.00143552 ... 0.04383017 0.01142565 0.11425767]
  ...
  [0.3741923  0.0468876  0.01168771 ... 0.24628015 0.07984894 0.13409954]
  [0.32618454 0.07154732 0.03341741 ... 0.34930819 0.10633796 0.15403793]
  [0.291583   0.09454725 0.04412511 ... 0.38305027 0.1404542  0.17689389]]

 [[0.53582334 0.00630557 0.00159332 ... 0.05985847 0.01163411 0.13394847]
  [0.53544622 0.005374

## SOM Clasificación

In [ ]:
# Matriz de iguales dimensiones que la matriz de pesos para guardar en cada neurona el último patrón clasificado
mapa_clasificacion = [[[]for _ in range(lado_mapa)] for _ in range(lado_mapa)]
# Matriz bidimensional para guardar el número de patrones reconocido por cada neurona
mapa_activaciones = np.zeros((lado_mapa, lado_mapa))
# Matriz bidimensional para guardar, para las neuronas con activación > 0, la distancia media de todos los patrones de la clase con su vector de pesos
mapa_distancias = np.zeros((lado_mapa, lado_mapa))
error_c = 0.0
error_t = 0.0
clases = 0

def scatter(mat):
    fig = go.Figure(data=[go.Scatter3d(
        x=mat[0],
        y=mat[1],
        z=mat[2],
        mode='markers',
        marker=dict(size=4, color='blue', colorscale='Viridis', opacity=0.8),
        text=[f'{df.at[i,"CountryName"]} clasificado en la neurona: ({mat[0,i]:.2f}, {mat[1,i]:.2f})' for i in range(n_paises)],
        hoverinfo='text')])
    fig.update_layout(scene=dict(
        xaxis_title='Lado Mapa',
        yaxis_title='Lado Mapa',
        zaxis_title='Identificador del País'
    ))
    pio.show(fig)


@jit
def histograma_3D(m_activaciones):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x_data, y_data = np.meshgrid( np.arange(m_activaciones.shape[1]), np.arange(m_activaciones.shape[0]))
    x_data = x_data.flatten()
    y_data = y_data.flatten()
    z_data = m_activaciones.flatten()
    ax.bar3d( x_data, y_data, np.zeros(len(z_data)), 1, 1, z_data)
    plt.show()

@jit
def patron_2_pais(dato):
  dat = np.array(dato)
  for i in range(0, n_paises):
    if (dat == datos[i]).all():
      return i
@jit
def entrada_2_pais(pat):
  patterns = np.zeros(n_paises)
  for i in range(0, len(pat)):
    patterns[patron_2_pais(pat[i])] = 1
  return patterns

@jit
def entradas_2_pais(m_clas):
  m_entradas = np.zeros((lado_mapa, lado_mapa, n_paises))
  for i in range(0, len(m_clas)):
    for j in range(0, len(m_clas)):
      if len(m_clas[i][j]) != 0:
        m_entradas[i][j] = entrada_2_pais(m_clas[i][j])
      else:
        m_entradas[i][j] = np.zeros(n_paises)
  return np.array(np.where(m_entradas == 1))


def clasificar_SOM(patrones, mat_pesos, m_clasificacion, m_activaciones, m_distancias, e_cuan, e_topo, n_clases):
    for i in range(0,n_paises):
        bmu_v, bmu_coor = calcular_bmu(patrones[i], mat_pesos, lado_mapa)
        #print(bmu_coor)
        m_clasificacion[bmu_coor[0]][bmu_coor[1]].append(patrones[i])
        m_activaciones[bmu_coor[0], bmu_coor[1]] += 1
        m_distancias[bmu_coor[0], bmu_coor[1]] += conseguir_distancia_euclidea(np.array(bmu_v), np.array(patrones[i]))
    e_cuan = np.sum(m_distancias)/n_paises
    e_topo = err_topologico(n_paises, datos, matriz_pesos, lado_mapa, n_caracteristicas)
    #m_clasificacion = np.asarray(m_clasificacion)
    m_distancias = m_distancias / m_activaciones
    m_distancias[~np.isfinite(m_distancias)] = 0
    n_clases = np.count_nonzero(m_activaciones)

In [ ]:
clasificar_SOM(datos, matriz_pesos, mapa_clasificacion, mapa_activaciones, mapa_distancias, error_c, error_t, clases)
#print(clases)
#print(mapa_clasificacion)
#print(mapa_activaciones)
#histograma_3D(mapa_activaciones)
#print(mapa_distancias)
#print(error_c)
#print(error_t)
m_venn = entradas_2_pais(mapa_clasificacion)
print(m_venn.shape)
scatter(m_venn)

(3, 171)


## SOM Prueba


# Anexos
## Tecnologías utilizadas

+ [Anaconda's Python 3.9](https://www.anaconda.com/products/individual "Versión de Python")
+ [Anaconda's Numpy](https://anaconda.org/anaconda/numpy "Librería Numérica")
+ [Anaconda's Matplotlib](https://anaconda.org/conda-forge/matplotlib "Librería Gráfica")
+ [Anaconda's Pandas](https://pandas.pydata.org/docs/index.html "Librería para DataFrames")
+ [Jupyter](https://jupyter.org/ "Editor a utilizar")

## Referecias

- https://medium.com/@abhinavr8/self-organizing-maps-ff5853a118d4#:~:text=Best%20Matching%20Unit%20is%20a,shortest%20distance%20is%20the%20winner.
- https://stats.stackexchange.com/questions/179026/objective-function-cost-function-loss-function-are-they-the-same-thing
- https://stackoverflow.com/questions/7872099/why-is-there-a-need-for-the-number-of-iterations-in-the-self-organizing-map
- https://www.cs.hmc.edu/~kpang/nn/som.html#algor
- https://www.geeksforgeeks.org/self-organising-maps-kohonen-maps/
- https://en.wikipedia.org/wiki/Self-organizing_map
- https://towardsdatascience.com/kohonen-self-organizing-maps-a29040d688da

### Plotting

- https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389